In [1]:
from seeq import spy
import pandas as pd

# spy.push

Uploads signals, conditions, scalars, and assets to the Seeq Server.

There are two main types of information processed by Seeq: _Data_ and _metadata_:

- **Data** is the time series and time interval information that is either collected or derived from sensor data. It consists of timestamps and values (samples), or time intervals and properties (capsules). This data can be plotted on a trend or used to train a neural network, for example.

- **Metadata** is the information about the data, that is independent of a particular point in time or time interval. For example, a signal's _name_, _description_ and _unit of measure_ is classified as metadata; or the formula that is used to derive a new signal from one or more source signals; or the asset tree that is used to model similar equipment or industrial processes.

The `spy.push()` command allows you to upload both types of information to Seeq Server. When you push _metadata_, you make entries in Seeq's data index, which allows you or other users to search for and find such entries. If you also push _data_, then samples or capsules or scalars will appear when the user selects those index entries for inclusion on a trend, scatter plot, scorecard or other visualization.

```
spy.push(data=None, metadata=None, item_type=None, workbook='Data Lab >> Data Lab Analysis',
         worksheet='From Data Lab', datasource=None, archive=False, type_mismatches='raise', errors='catalog')
```

Log into Seeq Server if you're not using Seeq Data Lab:

In [2]:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key')

""


## Workbook scoping

When you push any type of data, it is only available/discoverable within the workbook specified by the `workbook` argument. This allows you to _sandbox_ your activity by default, and only publish to your colleagues later when your experimentation is largely over.

## Pushing signal data

The simplest activity you can do with the `spy.push()` command is to read in a CSV file using Pandas and push it into Seeq. It will be stored in Seeq's internal time series database.

In [3]:
import csv
csv_file = pd.read_csv('csv_import_example.csv', parse_dates=['TIME(unitless)'], index_col='TIME(unitless)')
csv_file.head()

,BITDEP(ft),BLOCKCOMP(ft),DEP_RTN(ft),DEPTH(ft),FLOWIN(USgal/min),FLOWOUTPC(%),ROP_AVG(ft/h),DIFP(psi)
TIME(unitless),,,,,,,,
2018-07-25 23:00:01-06:00,4630.217,74.813,4627.0,4630.217,799.857,40.799,142.698,454.214
2018-07-25 23:00:03-06:00,4630.354,74.735,4627.5,4630.354,799.857,40.576,142.698,481.383
2018-07-25 23:00:05-06:00,4630.470,74.619,4627.5,4630.470,799.857,39.929,142.698,487.502
2018-07-25 23:00:07-06:00,4630.588,74.501,4627.5,4630.588,799.857,40.305,210.705,561.480
2018-07-25 23:00:09-06:00,4630.699,74.390,4627.5,4630.699,799.292,40.245,210.705,610.968


When you want to push data, it must have an index with a datetime data type. That's why we used the `parse_dates` and `index_col` arguments for `Pandas.read_csv()`.

Now you can just push it into Seeq:

In [4]:
push_results = spy.push(data=csv_file)
push_results

,Count,Result,Time
BITDEP(ft),11485,Success,00:00:03.01
BLOCKCOMP(ft),11485,Success,00:00:03.15
DEP_RTN(ft),11485,Success,00:00:03.49
DEPTH(ft),11485,Success,00:00:03.79
FLOWIN(USgal/min),11485,Success,00:00:04.10
FLOWOUTPC(%),11485,Success,00:00:04.17
ROP_AVG(ft/h),11485,Success,00:00:04.30
DIFP(psi),11485,Success,00:00:04.63


,Push Count,Push Time,Push Result,ID,Type,Name
BITDEP(ft),11485.0,0:00:03.005002,Success,98F25CFE-2A15-486D-93FB-DBADF802C5A4,StoredSignal,BITDEP(ft)
BLOCKCOMP(ft),11485.0,0:00:03.154502,Success,3DA6E58E-B8EC-49A4-B15A-857A08A08E94,StoredSignal,BLOCKCOMP(ft)
DEP_RTN(ft),11485.0,0:00:03.489495,Success,6F7BE71A-3F90-4536-9800-96D6E923E052,StoredSignal,DEP_RTN(ft)
DEPTH(ft),11485.0,0:00:03.788497,Success,26C08802-25A9-44A5-975C-C7808D04C097,StoredSignal,DEPTH(ft)
FLOWIN(USgal/min),11485.0,0:00:04.095000,Success,9B497841-E418-4E02-AD56-570CD4FCE607,StoredSignal,FLOWIN(USgal/min)
FLOWOUTPC(%),11485.0,0:00:04.165003,Success,E5443385-2F28-4AEB-AD11-AF4AD4834E13,StoredSignal,FLOWOUTPC(%)
ROP_AVG(ft/h),11485.0,0:00:04.302994,Success,4AFD33FD-230E-46FC-B4F4-7CB8F4D9A4E6,StoredSignal,ROP_AVG(ft/h)
DIFP(psi),11485.0,0:00:04.626998,Success,1ED1C41B-D89E-4EC8-B184-0C6D64DB9FAD,StoredSignal,DIFP(psi)


NOTE: Pushing data can be relatively slow. This is an area that Seeq will be optimizing in future versions.

You can push multiple times, and as long as the names are the same and the workbook hasn't changed, you'll just add to the existing signal.

## Pushing metadata

Now let's try pushing just metadata. You can see that the column names from the CSV file contain the unit of measure in parentheses. Let's use Pandas to extract the name and the unit of measure as separate columns.

In [5]:
better_metadata = push_results.copy()
better_metadata['Original Name'] = better_metadata.index
better_metadata['Name'] = better_metadata['Original Name'].str.extract(r'(.*)\(')
better_metadata['Value Unit Of Measure'] = better_metadata['Original Name'].str.extract(r'.*\((.*)\)')
better_metadata

,Push Count,Push Time,Push Result,ID,Type,Name,Original Name,Value Unit Of Measure
BITDEP(ft),11485.0,0:00:03.005002,Success,98F25CFE-2A15-486D-93FB-DBADF802C5A4,StoredSignal,BITDEP,BITDEP(ft),ft
BLOCKCOMP(ft),11485.0,0:00:03.154502,Success,3DA6E58E-B8EC-49A4-B15A-857A08A08E94,StoredSignal,BLOCKCOMP,BLOCKCOMP(ft),ft
DEP_RTN(ft),11485.0,0:00:03.489495,Success,6F7BE71A-3F90-4536-9800-96D6E923E052,StoredSignal,DEP_RTN,DEP_RTN(ft),ft
DEPTH(ft),11485.0,0:00:03.788497,Success,26C08802-25A9-44A5-975C-C7808D04C097,StoredSignal,DEPTH,DEPTH(ft),ft
FLOWIN(USgal/min),11485.0,0:00:04.095000,Success,9B497841-E418-4E02-AD56-570CD4FCE607,StoredSignal,FLOWIN,FLOWIN(USgal/min),USgal/min
FLOWOUTPC(%),11485.0,0:00:04.165003,Success,E5443385-2F28-4AEB-AD11-AF4AD4834E13,StoredSignal,FLOWOUTPC,FLOWOUTPC(%),%
ROP_AVG(ft/h),11485.0,0:00:04.302994,Success,4AFD33FD-230E-46FC-B4F4-7CB8F4D9A4E6,StoredSignal,ROP_AVG,ROP_AVG(ft/h),ft/h
DIFP(psi),11485.0,0:00:04.626998,Success,1ED1C41B-D89E-4EC8-B184-0C6D64DB9FAD,StoredSignal,DIFP,DIFP(psi),psi


In [6]:
spy.push(metadata=better_metadata)

,Asset,Condition,Overall,Relationship,Scalar,Signal
Items pushed,0,0,8,0,0,8


,Push Count,Push Time,ID,Type,Name,Original Name,Value Unit Of Measure,Push Result
BITDEP(ft),11485.0,0:00:03.005002,98F25CFE-2A15-486D-93FB-DBADF802C5A4,StoredSignal,BITDEP,BITDEP(ft),ft,Success
BLOCKCOMP(ft),11485.0,0:00:03.154502,3DA6E58E-B8EC-49A4-B15A-857A08A08E94,StoredSignal,BLOCKCOMP,BLOCKCOMP(ft),ft,Success
DEP_RTN(ft),11485.0,0:00:03.489495,6F7BE71A-3F90-4536-9800-96D6E923E052,StoredSignal,DEP_RTN,DEP_RTN(ft),ft,Success
DEPTH(ft),11485.0,0:00:03.788497,26C08802-25A9-44A5-975C-C7808D04C097,StoredSignal,DEPTH,DEPTH(ft),ft,Success
FLOWIN(USgal/min),11485.0,0:00:04.095000,9B497841-E418-4E02-AD56-570CD4FCE607,StoredSignal,FLOWIN,FLOWIN(USgal/min),USgal/min,Success
FLOWOUTPC(%),11485.0,0:00:04.165003,E5443385-2F28-4AEB-AD11-AF4AD4834E13,StoredSignal,FLOWOUTPC,FLOWOUTPC(%),%,Success
ROP_AVG(ft/h),11485.0,0:00:04.302994,4AFD33FD-230E-46FC-B4F4-7CB8F4D9A4E6,StoredSignal,ROP_AVG,ROP_AVG(ft/h),ft/h,Success
DIFP(psi),11485.0,0:00:04.626998,1ED1C41B-D89E-4EC8-B184-0C6D64DB9FAD,StoredSignal,DIFP,DIFP(psi),psi,Success


## Pushing condition and capsule data

You can also push capsules, which are time intervals of interest, to Seeq by supplying a DataFrame with `Capsule Start` and `Capsule End` columns. Any additional columns will be added as properties of the capsule.

In [7]:
capsule_data = pd.DataFrame([{
    'Capsule Start':    pd.to_datetime('2019-01-10T09:00:00.000Z'),
    'Capsule End':      pd.to_datetime('2019-01-10T17:00:00.000Z'),
    'Operator On Duty': 'Mark'
}, {
    'Capsule Start':    pd.to_datetime('2019-01-11T09:00:00.000Z'),
    'Capsule End':      pd.to_datetime('2019-01-11T17:00:00.000Z'),
    'Operator On Duty': 'Hedwig'
}])

capsule_data

,Capsule End,Capsule Start,Operator On Duty
0,2019-01-10 17:00:00+00:00,2019-01-10 09:00:00+00:00,Mark
1,2019-01-11 17:00:00+00:00,2019-01-11 09:00:00+00:00,Hedwig


When you push capsule data, you must supply a `metadata` DataFrame that is exactly 1 row and contains, at minimum, the `Name`, `Type` and `Maximum Duration` columns like so:

In [8]:
spy.push(data=capsule_data,
         metadata=pd.DataFrame([{
             'Name': 'Operator Shifts',
             'Type': 'Condition',
             'Maximum Duration': '2d'
         }]))

,ID,Name,Count,Time,Result
0,062CC7B3-A58D-4BF0-962E-33CCB6FE564B,Operator Shifts,2,00:00:00.09,Success


,Maximum Duration,Name,Type,Push Result,Datasource Class,Datasource ID,Data ID,ID,Push Count,Push Time
0,2d,Operator Shifts,StoredCondition,Success,Seeq Data Lab,Seeq Data Lab,[ECB02558-08DF-47F2-9B75-536753690B60] {Condit...,062CC7B3-A58D-4BF0-962E-33CCB6FE564B,2,0:00:00.089999
